In [3]:
# !pip install couchdb
# !pip install tweepy==v4.5.0 

# https://codeleading.com/article/24561506756/
# https://gist.github.com/marians/8e41fc817f04de7c4a70

In [4]:
"""
ssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com

sudo yum update -y
sudo yum install -y docker

sudo service docker start
sudo docker pull couchdb
sudo docker run -d --name my-couchdb -p 5984:5984 -e COUCHDB_USER='admin' -e COUCHDB_PASSWORD='123456' couchdb:latest

logout

ssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com



——————————————————

ssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com
——
sudo service docker start

logut
——
ssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com


http://localhost:15984/_utils

"""

'\nssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n\nsudo yum update -y\nsudo yum install -y docker\n\nsudo service docker start\nsudo docker pull couchdb\nsudo docker run -d --name my-couchdb -p 5984:5984 -e COUCHDB_USER=\'admin\' -e COUCHDB_PASSWORD=\'123456\' couchdb:latest\n\nlogout\n\nssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n\n\n\n——————————————————\n\nssh -i "CCC.pem" ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n——\nsudo service docker start\n\nlogut\n——\nssh -i CCC.pem -L localhost:15984:172.17.0.2:5984 -N -f ec2-user@ec2-3-104-118-191.ap-southeast-2.compute.amazonaws.com\n\n\nhttp://localhost:15984/_utils\n\n'

In [5]:
# couchbd_settings
address = 'localhost:15984'
username = 'admin'
password = '123456'
tweet_db = 'raw_tweets'

In [6]:
import couchdb
# from couchbd_settings import *

# use authenticated connection to connect to couchdb server for writing to the database
def db_connect(dbname):

    couchserver = couchdb.Server('http://' + username + ':' + password + '@' + address)
    try:
      db = couchserver[dbname]
    except:
      db = couchserver.create(dbname)

    return db

db = db_connect(tweet_db)

def save_to_db(tweet, dbname):
  db.save(tweet)

In [7]:
# api_credentials

# Weimin 500k cap
# CONSUMER_KEY = '2Qgue4p6wHH3dopq2ztD2ABos'
# CONSUMER_SECRET = 'JwSn0M1oYD8OSkbO2KU3uxAPWUCwiEr4GOrSSJDKvXjCbgXynt'
# ACCESS_TOKEN = '1510262166045458432-jGTjHomcIQzCmNO4qE73ByiicZJUD2'  
# ACCESS_TOKEN_SECRET = '3nYPGi7zbw2Z9LiuKqR1tzGsepuO1y8m1L5zISjIj0BqQ'
# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAAoxbAEAAAAApib47uKVACiyXfOSAj8BELeJaf4%3DT4L0eLVTNooQRZVftkppGGnEoRqau3K2R73ZJYo3jB8TZFghZJ'

# Wen 2m cap
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAGjcbAEAAAAANc0%2BWz6ZlsVZCZLEHegK9luuZXo%3DBKAeeJtUU6tKm3YqNHGz8u4jEfvDhTsQ9LnUq5Sz1GofEBI3mv'
ACCESS_TOKEN = '1956431324-p8F2IRC56igWyDBPdyLyENv1hpWzurywGDR65Vu'
ACCESS_TOKEN_SECRET = 'H527uwZmSQAw5em5HavBKRScQ36Fw1siO5GimhU4vX0kZ'
CONSUMER_KEY = 'FDYIQR8qhQPl2fS61IH3ptRv5'
CONSUMER_SECRET = 'UycadDkLnlDZGPev27mDBtyfHd39Eyk6psLVPpzH5zPvP4eE7D'


In [8]:
import tweepy
# from api_credentials import *

def authenticate():

  client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

  return client

client = authenticate()


In [9]:
"""
def query(query_item):
  query = 

return query

"""

query = 'corona OR coronavirus OR covid-19 -is:retweet'

In [10]:
import json

def search_recent_tweets(client, query, dbname):

    # start_time = ''
    # end_time = ''

    for response in tweepy.Paginator(client.search_recent_tweets, 
                                query = query, 
                                tweet_fields = ['created_at', 'geo', 'lang'],
                                place_fields = ['geo'],
                                expansions = 'geo.place_id').flatten(1): #delete number if not testing

        try:
            tweet = json.loads(json.dumps(response.data))
            if tweet.get('id') not in dbname:
                save_to_db(tweet, dbname)
        except Exception as e:
            print(e)
            pass
        
search_recent_tweets(client, query, tweet_db)

In [11]:
# 'great_mel': [144.33363404800002, -38.50298801599996, 145.8784120140001, -37.17509899299995]

# if tweet['geo'].get('coordinates', None) is not None:
#     coordinates = tweet['geo']['coordinates']['coordinates']
#     print(coordinates)

In [ ]:
import json

class TwitterStreamer(tweepy.Stream):

    def on_status(self, status):
        print(f'status_text: {status.text}')

    def on_data(self, data):
        tweet = json.loads(data)
        if (tweet.get('retweeted_status', None) is None) and (tweet.get('id_str') not in tweet_db):
            print(tweet)
            save_to_db(tweet, tweet_db)
    
    def on_error(self, status):
        print(status)
        
    def on_exception(self, exception):
        print(f'exception: {exception}')
        return

stream = TwitterStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream.filter(track = ['corona', 'coronavirus', 'covid-19'])
        stream.sample(tweet_fields = ['created_at', 'geo', 'lang', 'retweeted_status'],
                      place_fields = ['geo'],
                      expansions = 'geo.place_id')
    except:
        continue

{'created_at': 'Fri Apr 08 14:05:36 +0000 2022', 'id': 1512431470270459907, 'id_str': '1512431470270459907', 'text': '@LolaGanzarolli @patbhamilton @EmmBezas @ToniaBuxton Absolutely to all of that \nThere was no pandemic, not even clo… https://t.co/lmdlr3VutJ', 'display_text_range': [53, 140], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': True, 'in_reply_to_status_id': 1512386389438050306, 'in_reply_to_status_id_str': '1512386389438050306', 'in_reply_to_user_id': 1225736772, 'in_reply_to_user_id_str': '1225736772', 'in_reply_to_screen_name': 'LolaGanzarolli', 'user': {'id': 999746018, 'id_str': '999746018', 'name': 'paint it black', 'screen_name': 'CollinsMikeyc', 'location': None, 'url': None, 'description': 'Love my family & 3 dogs - Manchester England #KBF', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 4680, 'friends_count': 4858, 'listed_count': 65, 'favourites_count': 6289, 'stat

In [ ]:
# import tweepy
# # from twitter_credentials import *
# from tweepy.streaming import StreamListener
# from tweepy import OAuthHandler
# from tweepy import Stream

# def authenticate():

#   auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
#   auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#   return tweepy.API(auth, wait_on_rate_limit = True)

# api = authenticate()